In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import csv
import re
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def zapisi_u_csv(podaci, path):
    file_postoji = os.path.isfile(path)

    with open(path, 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        if not file_postoji:
            writer.writerow(['Word', 'Description'])

        writer.writerows(podaci)

In [ ]:
neologizmi = [
    ['nekapnica', 'naprava koja se utakne u grlić vinske boce radi sprječavanja kapanja vina pri nalijevanju'],
    ['borkinja', 'koja vrši vojničku službu'],
    ['borkinja', 'žena borac za ljudska i ženska prava'],
    ['borkinja', 'koja se bavi borilačkim sportom'],
    ['brzogriz', 'brza hrana'],
    ['ispraznica', 'floskula'],
    ['golijada', 'velik broj zabijenih golova na nekoj nogometnoj utakmici'],
    ['gmazara', 'nastamba za gmazove'],
    ['estradnjak', 'osoba poznata javnosti, najčešće glumci ili pjevači'],
    ['drhtalica', 'desert koji se priprema kuhanjem voća, a hlađenjem se pretvara u želatiniziranu masu'],
    ['dopusnica', 'dokument koji osobi omogućava nešto'],
    ['dodirnik', 'ekran s touch screen tehnologijom'],
    ['dlanovnik', 'digitalni prijenosni uređaj koji stane u dlan korisnika'],
    ['kartičar', 'osoba koja posluje s kreditnim karticama'],
    ['lisinskizam', 'koji se odnosi na ljubav prema glazbi, prema imenu koncertne dvorane Vatroslav Lisinski'],
    ['lisinskist', 'koji podupire ili prati programe koncernte dvorane Vatroslav Lisinski'],
    ['bonaš', 'koji koristi mobilni telefon na bonove, tj. uz povremenu nadoplatu'],
    ['bonaš', 'koji ne koristi pretplatu'],
    ['zmajada', 'organizirano natjecanje ili igranje papirnatim zmajevima'],
    ['zatipak', 'pogreška u pisanju, tipkanju i sl.'],
    ['vatrozid', 'računalni sustav kojim se štiti neko računalo ili mrežu od nedopuštenoga pristupa no ostavlja se mogućnost komuniciranja'],
    ['vatrozid', 'firewall'],
    ['raskružje', 'hrvatski prijedlog za kružni tok i rotor'],
    ['raskolnica', 'bijela linija na cesti koja dijeli kolnik'],
    ['obiteljaš', 'koji dijeli svjetonazor i stavove koje propagira udruga U ime obitelji'],
    ['naplatnica', 'naplatna kućica, kućica za naplatu cestarine na autocestama'],
    ['napetica', 'knjiga ili film koji drže čitatelja ili gledatelja u neizvjesnosti, napetosti'],
    ['napetica', 'zanimljiv film, knjiga, događaj'],
    ['ljubionica', 'klupa za zaljubljene parove'],
    ['čokoprofesionalac', 'koji se bavi proizvodnjom čokolade'],
    ['čokoprofesionalac', 'koji pravi slastice od čokolade'],
    ['batinaš', 'koji tuče (batina) druge'],
    ['alternativka', 'ženska osoba koja pripada alternativi, drugome, drugačijemu'],
    ['aktualac', 'aktualni sat, aktualno prijepodne koje se održava na početku svake sjednice Sabora prilikom kojega zastupnici mogu pisanim ili usmenim putem postavljati pitanja članovima Vlade na koja su ovi dužni odgovoriti'],
    ['ponovak', 'repriza'],
    ['bocar', 'sakupljač boca'],
    ['rječarenje', 'rafting'],
    ['biciklirati', 'voziti bicikl'],
    ['kliznica', 'slajd'],
    ['daroteka', 'gift shop'],
    ['rasprodajnica', 'outlet'],
    ['osjećajnik', 'emotikon'],
    ['osjećajnik', 'emoji'],
    ['smećnjak', 'kanta za smeće'],
    ['dvokriška', 'sendvič'],
    ['rabljenarnica', 'second hand shop'],
    ['utjecatelj', 'influencer'],
    ['trogodak', 'tri uzastopna zgoditka, pogotka'],
    ['promopisac', 'copywriter'],
    ['ubrusnik', 'držač za ubruse'],
    ['vodomet', 'vaterpolo'],
    ['konačnik', 'rezultat'],
    ['kuterac', 'udarac iz kuta'],
    ['poslovlje', 'biznis'],
    ['kovaničnik', 'novčanik za kovanice'],
    ['napucnjak', 'oštri udarac u sportu'],
    ['plovopis', 'književna djela o putovanjima brodom'],
    ['sigurnik', 'sef'],
    ['reciklana', 'reciklažno postrojenje'],
    ['tipkomamac', 'clickbait'],
    ['čvornik', 'hub'],
    ['uglednina', 'goodwill'],
    ['namrežno', 'online'],
    ['bakroza', 'wilsonova bolest'],
    ['dimodojavnik', 'detektor dima'],
    ['izazor', 'retrovizor'],
    ['obrubnik', 'lajsna'],
    ['odricajnica', 'izjava o odricanju od odgovornosti'],
    ['otpadalište', 'odlagalište otpada'],
    ['prestrujnik', 'strujni adapter'],
    ['pridatak', 'plug in'],
    ['prijevodnici', 'titlovi'],
    ['prikaznik', 'displej'],
    ['šećerice', 'biljke od kojih se proizvodi šećer'],
    ['zaključaj', 'lockdown'],
    ['znojilište', 'sauna'],
    ['novotvorenica', 'neologizam'],
    ['čavrljanje', 'chat'],
    ['samoizolacija', 'self isolation'],
    ['sebić', 'selfie'],
    ['zbiljokaz', 'reality show'],
    ['nazovnik', 'nominativ'],
    ['rodnik', 'genitiv '],
    ['datnik', 'dativ'],
    ['tvornik', 'akuzativ'],
    ['zovnik', 'vokativ'],
    ['mjestnik', 'lokativ'],
    ['orudnik', 'instrumental'],
    ['očvrsje', 'hardver'],
    ['uručak', 'ono što se dijeli kao radni materijal'],
    ['osjetnik', 'senzor'],
    ['knjigopis', 'popis literature'],
    ['dalekovidnica', 'televizija'],
    ['uspornik', 'ležeći policajac'],
    ['sitnoval', 'mikrovalna pećnica'],
    ['zatipak', 'pogreška u tipkanju zbog brzine tipkanja'],
    ['zatipak', 'tipfeler'],
    ['zatipak', 'typo'],
    ['predočnik', 'ekran'],
    ['iskočnik', 'pop-up window'],
    ['iskočnik', 'prozor koji se iznenada pojavi na zaslonu računala'],
    ['proširenica', 'dodatak računalnom programu koji mu povećava funkcionalnost'],
    ['proširenica', 'extension'],
    ['milozvuk', 'podcast']
]

zapisi_u_csv(neologizmi, '/content/drive/MyDrive/korpus_neologizmi.csv')

In [ ]:
def ocisti_i_razdvoji(text):
    markeri = r'[\[\]〈〉;]'
    temp_text = re.sub(markeri, '|', text)

    dijelovi = temp_text.split('|')

    # Očisti svaki dio (makni višak razmaka) i zadrži samo one koji su prave rečenice
    definicije = [d.strip() for d in dijelovi if len(d.strip()) > 5]

    definicije = [re.sub(r'[\(\[〈][ ,]*[\)\]〉]', '', d) for d in definicije]
    definicije = [re.sub(r'[\[〈]', '', d) for d in definicije]
    definicije = [re.sub(r'[\]〉]', '', d) for d in definicije]
    definicije = [re.sub(r' ,[ ,]*', ', ', d) for d in definicije]
    definicije = [d.lstrip(',').rstrip(',').strip() for d in definicije]
    definicije = [d.strip() for d in definicije if len(d.strip()) > 5]

    return definicije

def ukloni_naglaske(text):
    # Mapa naglašenih slova u obična slova (čuvamo č, ć, š, ž, đ)
    zamjene = {
        'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a', 'ȁ': 'a', 'ȃ': 'a', 'ā': 'a',
        'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e', 'ȅ': 'e', 'ȇ': 'e', 'ē': 'e',
        'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i', 'ȉ': 'i', 'ȋ': 'i', 'ī': 'i',
        'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o', 'ȍ': 'o', 'ȏ': 'o', 'ō': 'o',
        'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u', 'ȕ': 'u', 'ȗ': 'u', 'ū': 'u',
        'ŕ': 'r', 'ȑ': 'r', 'ȓ': 'r'
    }

    # Prolazimo kroz tekst i mijenjamo samo ako je u mapi
    for naglaseno, obicno in zamjene.items():
        text = text.replace(naglaseno, obicno)

    # Dodatno: uklanjanje kombiniranih unicode naglasaka
    # Ovaj dio briše 'combining characters' ali pazimo da ne obriše bazu
    text = re.sub(r'[\u0300-\u036f]', '', text)

    return text

In [ ]:
def scrape_all(path):
    base_url = "https://rjecnik.hr/?letter="
    letter = ['a','b','c','č','ć','d','dž','đ','e','f','g','h','i','j','k','l','lj','m','n','nj','o','p','r','s','š','t','u','v','z','ž']
    page_num = 1
    i = 0
    podaci_za_slovo = []
    while True:
        print(f"Stranica {page_num}...")
        response = requests.get(base_url + letter[i] + "&page=" + str(page_num))

        # Ako stranica više ne postoji (404), gotovi smo
        if response.status_code != 200:
            break

        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        trs = soup.find_all('tr')

        # Ako je trs prazan, gotovi smo sa trenutnim slovom,
        # prelazimo na sljedeće ili ako smo obradili slovo ž stajemo
        if not trs:
            if podaci_za_slovo:
                zapisi_u_csv(podaci_za_slovo, path)
                print(f"✅ Slovo {letter[i]} je spremljeno")
                podaci_za_slovo = []
            if letter[i] == 'ž':
                break
            i += 1
            page_num = 0
        page_num += 1

        for tr in trs:
            title_tag = tr.find('span', class_='word')
            content_div = tr.find('div', class_='description')

            if title_tag and content_div:
                rijec = ukloni_naglaske(title_tag.get_text(strip=True).lower())
                rijec = re.sub(r'[.¹²]', '', rijec)

                if len(rijec) > 2:
                    d_tag = content_div.find_all('span', class_='Normala')
                    if d_tag:
                        opis = ' '.join([d.get_text(strip=True) for d in d_tag])
                    else:
                        opis = content_div.get_text(strip=True)
                    opis = ocisti_i_razdvoji(opis)
                    if ' ' not in rijec and '-' not in rijec:
                        for o in opis:
                            podaci_za_slovo.append((rijec, o))
        time.sleep(1)



In [ ]:
korpus = '/content/drive/MyDrive/korpus.csv'
finalni_podaci = scrape_all(korpus)

train_df = pd.read_csv(korpus)
sva_slova = sorted(list(set(''.join(train_df['Word'].astype(str).tolist()))))
print(f"Rječnik: {sva_slova}")

In [ ]:
df_hjp = pd.read_csv(korpus)
df_neo = pd.read_csv('/content/drive/MyDrive/neologizmi.csv')

# Prvo odvajamo Train dio (40%)
train_neo, temp_neo = train_test_split(df_neo, test_size=0.6, random_state=42)
# Ostatak dijelimo na Dev i Test (pola-pola)
dev_neo, test_neo = train_test_split(temp_neo, test_size=0.5, random_state=42)

train_hjp, temp_hjp = train_test_split(df_hjp, test_size=0.2, random_state=42)
dev_hjp, test_hjp = train_test_split(temp_hjp, test_size=0.5, random_state=42)

train_final = pd.concat([train_hjp, train_neo]).sample(frac=1).reset_index(drop=True)
dev_final = pd.concat([dev_hjp, dev_neo]).sample(frac=1).reset_index(drop=True)
test_final = pd.concat([test_hjp, test_neo]).sample(frac=1).reset_index(drop=True)

train_final.to_csv('/content/drive/MyDrive/train.csv', index=False)
dev_final.to_csv('/content/drive/MyDrive/dev.csv', index=False)
test_final.to_csv('/content/drive/MyDrive/test.csv', index=False)

print(f"Podjela završena!\nTrain: {len(train_final)}\nDev: {len(dev_final)}\nTest: {len(test_final)}")